In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from IPython.display import display
import pandas as pd
import h5py
import os
# to load data from the test file
from utils.create_datasets import SumDatasets
# To load the data sets here, also we will create some examples to explore torch
data_dir = '../data/'
from utils import config
import pickle
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

/home/chenxiang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sum_dataset = SumDatasets(os.path.join(data_dir, 'features-600-40_v2.hdf5'))

In [3]:
features_1, features_2, features_3, features_4 = sum_dataset[0:10] # assume we have a batch data of 20 items.
display(features_2)

trainsformed finished


tensor([600, 600, 600, 600, 600, 600, 600, 600, 600, 436])

## given the data, build a very naive summarization model here
* first, we need to compute all the hidden state of each time step and a final state, contain cell state and hidden state of the final time and feed it into the network and do the decoder agian.

In [9]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.embed = nn.Embedding(config.NUM_WORDS + 2, config.embedding_dim)
        self.rnn = nn.LSTM(config.embedding_dim, config.hidden_dim, bidirectional=True)
        self.reduce_ = nn.Linear(2 * config.hidden_dim, config.hidden_dim)
        
    def forward(self, X, seq_lens):
        """return the final satets and also the outputs of each timesteps, for the later usage of
        computing the Attentaion matrix foe each time step input of the Decoder.
        
        Parameters
        ----------
        X : [Torch tensor with batch*MAX_STEP]
            
        seq_lens : [descend order of the real length of the data]
        """
        X = self.embed(X)
        batch_size = X.size()[0]
        packed_x = pack_padded_sequence(X, seq_lens, batch_first=True)
        outputs, hidden = self.rnn(packed_x)
        outputs, seq_lens = pad_packed_sequence(outputs, batch_first=True)
        # outputs is a bathc*max_enc_steps*(2*hidden_dim), but for the hidden
        # must give then to batch first format, so we need to implement this
        # with the following code.
        hidden_c, hidden_s = hidden
        hidden_c = self.reduce_(hidden_c.permute([1, 0, 2]).contiguous().view(batch_size, -1))
        hidden_s = self.reduce_(hidden_s.permute([1, 0, 2]).contiguous().view(batch_size, -1))
        
        hidden_c.unsqueeze_(1)
        hidden_s.unsqueeze_(1)
        # to make the hidden to be batch first and later, do the loop over time
        hidden = (hidden_c.permute([1,0,2]), hidden_s.permute([1, 0, 2]))
        return outputs, hidden

In [10]:
encoder = Encoder()
outputs, hidden = encoder(features_1, features_2)
display(hidden[0].shape)

torch.Size([1, 10, 50])

In [15]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.embed = nn.Embedding(config.NUM_WORDS+4, config.embedding_dim)
        self.rnn = nn.LSTM(config.embedding_dim, config.hidden_dim, batch_first=True)
        self.logits = nn.Linear(config.hidden_dim, config.NUM_WORDS, bias=False)
        
        
        
    def forward(self, X, hidden):
        """
        using the encoder's hidden state to initiaize the decoder
        input and also use the teaching force in the training mode. here
        TODO: how we tell the real difference between the train, teaching force and
        the evalutaion?
        """
        X = self.embed(X)
        # first we need to transoform the X to be the seq_len first
        X = X.unsqueeze_(1) # just one time step, the X now shoud be batch_size*1*embeeding_dim
        outputs, hidden = self.rnn(X, hidden)
        outputs = F.log_softmax(self.logits(outputs.squeeze_(1)), dim=1)
        return outputs, hidden

In [24]:
decoder = Decoder()
outputs = []
# simply test the time dimension 
# feature 3 shoud be time first
# features_4 = features_4.permute([1, 0])
loss = 0
critertion = nn.NLLLoss(ignore_index=0)
for di in range(config.max_dec_steps):
    X = features_3[di]
    output, hidden = decoder(X, hidden)
    # here to implement the loss wrt target
    target = features_4.permute(1, 0)[di]
    loss += critertion(output, target)

RuntimeError: Assertion `cur_target >= 0 && cur_target < n_classes' failed.  at /pytorch/aten/src/THNN/generic/ClassNLLCriterion.c:93

In [10]:
from torch.utils.data import DataLoader, SequentialSampler, BatchSampler
sampler_ = BatchSampler(SequentialSampler(sum_dataset), batch_size = 10, drop_last=False)

train_data = DataLoader(sum_dataset, shuffle=False, batch_sampler=sampler_)

In [11]:
for i_batch, sample_batched in enumerate(train_data):
    display(sample_batched)
    break

trainsformed finished
trainsformed finished
trainsformed finished
trainsformed finished
trainsformed finished
trainsformed finished
trainsformed finished
trainsformed finished
trainsformed finished
trainsformed finished


[tensor([[[   18,    20,     1,  ..., 50001,    51,     7]],
 
         [[ 4011, 26440,   315,  ...,    89,     5,   739]],
 
         [[   12,    18,  1212,  ...,    84,  1848,  1979]],
 
         ...,
 
         [[   14,    36,   263,  ...,  1709,   184,   282]],
 
         [[  208,     7,  4228,  ...,     0,     0,     0]],
 
         [[   52,   178,    79,  ...,     2,  1362,    40]]]),
 tensor([600, 600, 600, 600, 600, 600, 600, 600, 436, 600]),
 tensor([[[50003,    91,    75,     7,    31,  1336,  1351,    15,   623,     7,
            1336,   208,  1351,  1351,   623,     5,   215,   623,  1351,     7,
            1559,  1336,  1670,   623,   215,    15,  1307,    91,   149,    75,
             211,     0,     0,     0,     0,     0,     0,     0,     0,     0]],
 
         [[50003,  2441,  2250,     5,    31,    31,  1336,   215,  1336,     5,
            2017,    31,   623,  1351,  1710,  1351,     5,   215,   208,     5,
              31,    15,  1307,  1710,   770,  1336,  1